Etwas styling ...

In [ ]:
%%html
<style>
h1 { color: rgb(226, 0, 26); }
h2 { color: rgb(0, 80, 120); }
</style>

# BigData - DBWeather
---
*Programmentwurf - Wintersemester 2022 - DHBW Mannheim*\
Daniel Czeschner, Kai Grübener, Frederik Wolter

## 0. Inhalt

1. Business Understanding

2. Data Understanding

3. Data Preperation

4. Modeling

5. Evaluation

6. Deployment

*... in Anlehnung an CRISP-DM*

## 1. Business Understanding

* __Perspektive__: Deutsche Bahn

* __Vermutung__: 

<center><i>Klimaanlagen sorgen für Störungen bei extremen Temperaturen, besonders bei ICE's</i></center>

* __Ziel__: Erhebung & Auswertung von Daten zu Zugverspätungen & -ausfällen
  * Grundlegende statistische Darstellung von Fahrplan-Änderungs-Daten 
  * Erhebung von Wetterdaten für die Analyse potentieller Korrelationen
  * Identifizieren von Problemen und Ableiten möglicher Problemursachen

## 2. Data Understanding

Verfügbare Datenquellen:
  * Deutsche Bahn Timetable API<br> 
    `apis.deutschebahn.com/db-api-marketplace/apis/timetables/`
  
  * Deutsche Bahn Reiseauskunft API<br>
    `reiseauskunft.bahn.de/bin/`
  
  * BrightSky API<br>
    `api.brightsky.dev/`

## 2.1 Data Understanding – DB Timetable

* API des Deutschen Bahn API-Marketplace

* `/station/{pattern}`
```xml
<station meta="518342" name="Mannheim Hbf" eva="8000244" p="..." ...="" />
```

* `/fchg/{evaNo}`: viele schlecht dokumentiert Daten über Fahrplan-Änderungen
  * Zugtyp, Zugnummer
  * Durchsagen? 
  * **geänderte** Gleise, Halte, Ankünfte, Abfahrten, Ausfälle
    <div class="alert alert-danger">
        <b>Achtung:</b> keine Plan-Daten (andere API) -> ungeignet für Verspätungsbestimmung
    </div>

## 2.2 Data Understanding – DB Reiseauskunft

* API der Deutschen Bahn – _"not so official"_

* `bhftafel.exe/dn?L=vs_java&start=yes&boardType={arr|dep}&input={evaNo}`
  
    <pre><code class="" style="max-height: none;">    8000244 Mannheim Hbf
      17:23
      IC  2217
      + 35
      17:24
      S      1
      no
      17:31
      ICE  611
      cancel
      17:33
      ICE  42
      + 5</code></pre>

## 2.3 Data Understanding – BrightSky

* API des Deutschen Wetterdienstes

* `/weather?lat={Latitude}&lon={Longitude}&date={Date}&tz={TimeZone}`

    <pre><code class="cm-s-ipython language-json" style="max-height: none;">   <span class="cm-string">"weather"</span>: [{ 
                    <span class="cm-string cm-property">"timestamp"</span>: <span class="cm-string">"2022-11-03T00:00:00+01:00"</span>,
                    <span class="cm-string cm-property">"source_id"</span>: <span class="cm-number">184401</span>,
                    <span class="cm-string cm-property">"pressure_msl"</span>: <span class="cm-number">1021.7</span>,
                    <span class="cm-string cm-property">"sunshine"</span>: <span class="cm-number">0.0</span>,
                    <span class="cm-string cm-property">"temperature"</span>: <span class="cm-number">6.8</span>,     
                    <span class="cm-string cm-property">"condition"</span>: <span class="cm-string">"dry"</span>,
                    <span class="cm-string cm-property">"wind_speed"</span>: <span class="cm-number">4.3</span>,
                    <span class="cm-string cm-property">"cloud_cover"</span>: <span class="cm-number">100</span>,     
                    <span class="cm-string cm-property">"relative_humidity"</span>: <span class="cm-number">96</span>, <span class="cm-meta">...</span>
                  }, <span class="cm-meta">...</span> ],</code></pre>
    ```json
    "sources": [ ... ]
    ```

## 3. Data Preparation

__Abspeichern der Daten__

* Welche Daten speichern?

* Wie Daten speichern?

* In welchem Format Daten speichern?

## 3.1 Data Preparation – Deutsche Bahn

* Speichern als **einzelne** Ankunft/Abfahrt

* Hinzufügen von `eva`, `board_type` und `delay`

* Parsen der Informationen zu `JSON`:

    ```json
    {
        "timestamp": "2022-11-03T17:48:00+01:00",
        "eva": 8000105,
        "board_type": "arr",
        "con_line": "725",
        "con_type": "ICE",
        "delay": 26
    }
    ```
    
<div class="alert alert-block alert-info">
    <b>Info:</b> <code>delay</code> "cancel" wird als <code>-1</code> dargestellt
</div>
    

## 3.2 Data Preparation – BrightSky

* Speichern als **einzelne stündliche** Datensätze

* Hinzufügen von `lon` und `lat`
    <pre><code class="cm-s-ipython language-json" style="max-height: none;">    { 
        <span class="cm-string cm-property">"timestamp"</span>: <span class="cm-string">"2022-11-25T14:00:00+01:00"</span>,
        <span class="cm-string cm-property">"lat"</span>: <span class="cm-number">50.05</span>,        
        <span class="cm-string cm-property">"lon"</span>: <span class="cm-number">8.6</span>,
        <span class="cm-string cm-property">"cloud_cover"</span>: <span class="cm-number">88</span>,   
        <span class="cm-string cm-property">"condition"</span>: <span class="cm-string">"dry"</span>,
        <span class="cm-string cm-property">"pressure_msl"</span>: <span class="cm-number">1018.8</span>,
        <span class="cm-string cm-property">"relative_humidity"</span>: <span class="cm-number">83</span>,
        <span class="cm-string cm-property">"sunshine"</span>: <span class="cm-number">32</span>,
        <span class="cm-string cm-property">"temperature"</span>: <span class="cm-number">9.2</span>,
        <span class="cm-string cm-property">"wind_speed"</span>: <span class="cm-number">10.1</span>, 
        <span class="cm-meta">...</span> 
      }</code></pre>

## 4. Modeling

Die Pipeline ist wie folgt aufgebaut:
<br>
<br>
<center><img src="img/DataPipeline.png" /></center>

## 4.1 Modeling – Python

* __Database.py__ Skript als Wrapper für die benötigten Datenbankfunktionalitäten
 * Verbindungsaufbau zur MongoDB & Collection Variablen bereitstellen
 * upsert Funktion
 * Verbindung zur MongoDB trennen

## 4.1 Modeling – Python
### Das Laden der Daten exemplarisch für die Wetterdaten (vereinfacht):

In [ ]:
def _load_api_data(lat: float, lon: float, current_time: datetime.datetime) -> None:
    url = "https://api.brightsky.dev/weather"
    params = {
        "date": current_time.strftime("%Y-%m-%d"),
        "lon": lon,
        "lat": lat,
        "tz": "Europe/Berlin"
    }

    headers = {}
    r = requests.get(url=url, params=params, headers=headers, timeout=20)

    if r.status_code != 200:
        assert False

    answer = r.json()

    weather = answer['weather']

    for dataset in weather:
        dataset["lat"] = lat
        dataset["lon"] = lon
        _save_to_db(db=database, dataset=dataset)

## 4.1 Modeling – Python
### Das Speichern der Daten in die MongoDB (vereinfacht):

In [ ]:
def _save_to_db(db: Database, dataset: dict) -> None:
    keys = {
        "timestamp": dataset["timestamp"],
        "lat": dataset["lat"],
        "lon": dataset["lon"]
    }

    database.upsert(collection=db.mongo_data_weather, query=keys, update=dataset)

## 4.2 Modeling – Datenbank
* Für die Daten wurde ein MongoDB-Server unter Debian eingerichtet.
* Datenbank für dieses Projekt: DBWeather
 * __Collections:__
   * `data_train`: Daten der Deutschen Bahn
   * `data_weather`: Wetterdaten von Bright Sky

## 4.2 Modeling – Datenbank
* Die MongoDB wurde so konfiguriert, dass eine Authentifizierung der Clients notwendig ist.
 * Dadurch wird die Datenbank vor fremder Einsicht geschützt und konnte somit global verfügbar gemacht werden.
 * Zudem wurde ein Web-Dashboard aufgesetzt, worüber die gepspeicherten Daten einfach eingesehen werden können:
 
<center><img src="img/MongoDBDashboard.png" width="927" height="246"/></center>

## 4.3 Modeling – Ausfürhung

* Python-Skripte werden durch einen Debian-Server via `cronjob` ausgeführt
  * zur _halben_ und _vollen Stunde_
  
* Bei der Ausführung wird eine Log-Datei auf dem Server erstellt:

    <pre style="font-size: 0.8em !important;"><code style="max-height: none;">  2022-11-03 19:00:01,408 INFO    weather: Start main_weather execution ...
    2022-11-03 19:00:01,410 INFO    train: Start main_train execution ...
    2022-11-03 19:00:01,708 INFO    weather: Result lat=50.05 lon=8.6 has 25 elements
    2022-11-03 19:00:01,824 INFO    weather: Result lat=49.5 lon=8.48 has 25 elements
    2022-11-03 19:00:01,844 INFO    weather: finished: 0 inserted, 2 updated, 48 unchanged
    2022-11-03 19:00:01,844 INFO    weather: ###########################################
    2022-11-03 19:00:01,917 INFO    train: Result eva=8000105 type=arr has 690 lines, calc 230.0 sets
    2022-11-03 19:00:02,493 INFO    train: Result eva=8000105 type=dep has 672 lines, calc 224.0 sets
    2022-11-03 19:00:02,938 INFO    train: Result eva=8000244 type=arr has 291 lines, calc 97.0 sets
    2022-11-03 19:00:03,233 INFO    train: Result eva=8000244 type=dep has 264 lines, calc 88.0 sets
    2022-11-03 19:00:03,301 INFO    train: finished: 295 inserted, 119 updated, 225 unchanged
    2022-11-03 19:00:03,301 INFO    train: ###########################################</code></pre>

## 4.4 Modeling – Auswertung

* Für das Auswerten und Visualisieren der Daten wird dieses `Jupyter Notebook` verwendet
* Für die Präsentation wird die Extension [`Rise`](https://rise.readthedocs.io/en/stable/) verwendet. 

## 5. Evaluation
Benötigte Packages:

In [ ]:
from pandas import DataFrame
from database import Database

Verbindung zur mongoDB durch eignes Package `Database`:

In [ ]:
database = Database()
data_train = database.mongo_data_train
data_weather = database.mongo_data_weather

* Wetter kombis
    * temp <> verspätung (Zugart)
    * evtl. mehr

In [ ]:
# plot config
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.autolayout"] = True
mpl.rcParams["figure.figsize"] = [10.00, 4.00]

## 5.1 Evaluation – Wetter Einführung

In [ ]:
data_weather.count_documents({})

In [ ]:
result = data_weather.aggregate([
    {"$group":  { "_id": { "$dateToString": { "format": "%d.%m.%Y", "date": {"$toDate": "$timestamp"} } }, "count": {"$sum": 1} } },
    {"$sort": { "_id": 1 } } ])
plot = DataFrame(result).plot(x="_id", y="count", kind="line", xlabel="", style='.-')

## 5.2 Evaluation – Wetter

Wetter in Mannheim:

In [ ]:
result = data_weather.find({ "lat": 49.5, "lon": 8.48 })
plot = DataFrame(result).plot(x="timestamp", y="temperature", kind="line", xlabel="", style='.-')

In [ ]:
# beide in einem graphen? aber wie? 
result = data_weather.aggregate([
    {"$group":  { "_id": { "$dateToString": { "format": "%H:%M %d.%m.%Y", "date": {"$toDate": "$timestamp"} } }, "count": {"$sum": 1} } },
    {"$sort": { "_id": 1 } } ])
plot = DataFrame(result).plot(x="_id", y="count", kind="line", xlabel="", style='.-')

## 5.3 Evaluation – Züge Einführung

In [ ]:
data_train.count_documents(filter={})

In [ ]:
result = data_train.aggregate([
    {"$group": { "_id": { "$dateToString": { "format": "%d.%m.%Y", "date": {"$toDate": "$timestamp"} } }, "count": {"$sum": 1} } },
    {"$sort": { "_id": 1 } } ])
plot = DataFrame(result).plot(x="_id", y="count", kind="line", xlabel="", style='.-')

## 5.4 Evaluation – Zug-Arten

In [ ]:
result = data_train.aggregate([
    {"$group": { "_id": "$con_type", "count": {"$sum": 1} } },
    {"$sort": { "count": -1 } },
    {"$limit": 11 } ])
df = DataFrame(result)
plot = df.plot(x="_id", y="count", kind="pie", labels=df['_id'], legend=False, figsize=(3.5,10), autopct='%1.1f%%')

## 5.5 Evaluation – Delay

In [ ]:
result = data_train.aggregate([
    {"$group": { "_id": "$delay", "count": {"$sum": 1} } },
    {"$sort": { "count": 1 } } ])
plot = DataFrame(result).plot(x="_id", y="count", kind="line", xlabel="", style='.', logy=True)

## 5.6 Evaluation – Delay

In [ ]:
result = data_train.aggregate([
  {"$addFields": { "range": { "$switch": { "branches": [ { "case": { "$eq": [ "$delay", -1 ] }, "then": "nie" }, { "case": { "$eq": [ "$delay",  0 ] }, "then": "pünkltich"  }, { "case": { "$lte": [ "$delay", 5 ] }, "then": "<6" }],  "default": ">6" } } } },
  {"$group": { "_id": "$range", "count": { "$sum": 1 } } } ])
df = DataFrame(result)
plot = df.plot(x="_id", y="count", kind="pie", xlabel="", labels=df['_id'], legend=False, figsize=(3.5,10), autopct='%1.1f%%')

## 5.6 Evaluation – Delay ICE

In [ ]:
result = data_train.aggregate([
  {"$match": {"con_type": "ICE"} },
  {"$addFields": { "range": { "$switch": { "branches": [ { "case": { "$eq": [ "$delay", -1 ] }, "then": "nie" }, { "case": { "$eq": [ "$delay",  0 ] }, "then": "pünktlich"  }, { "case": { "$lte": [ "$delay", 5 ] }, "then": "<6" }],  "default": ">6" } } } },
  {"$group": { "_id": "$range", "count": { "$sum": 1 } } } ])
df = DataFrame(result)
plot = df.plot(x="_id", y="count", kind="pie", xlabel="", labels=df['_id'], legend=False, figsize=(3.5,10), autopct='%1.1f%%')

## 5.7 Evaluation – Walk of Shame

In [ ]:
DataFrame(data_train.find(sort=[("delay", -1)], filter={"board_type": "arr"}).limit(3))

... ohne Straßenbahnen ...

In [ ]:
DataFrame(data_train.find(sort=[("delay", -1)], filter={"board_type": "arr","con_type": {"$ne": "STR"}}).limit(3))

## 5.8 Evaluation – Züge + Temperatur

# TBD

## 6. Deployment

* Zusammenfassung: Beantwortung der Fragen aus Anwendungssicht
  * noch nicht genut Daten für zuverlässige Aussage
* (was hätte man anders machen können?)


### Probleme
* Der Server ist am 20.11.2022 eingefroren aus unbekannter Ursache.
* Dies wurde unglücklicherweise erst ungefähr zwei Tage Später festgestellt.
 * Die Wetter-Daten konnten hier nachträglich geladen werden.
 * Die DB-Daten jedoch nicht, da die API nicht die Daten der letzten Tage bereitstellt.

In [ ]:
database.close()

# Temp Test ===================================

In [ ]:
https://stackoverflow.com/a/33079582/13777031
https://rise.readthedocs.io/en/stable/customize.html
https://ourcodingclub.github.io/tutorials/pandas-time-series/